# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
import json
import azureml.core
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.model import Model
from azureml.core import Workspace, Experiment, Model, Dataset
from azureml.core.compute import ComputeTarget, AmlCompute
# from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
import os
import pandas as pd
import numpy as np
import json
import requests
import joblib
from sklearn.metrics import confusion_matrix
import itertools



## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.

I got this dataset from Kaggle. In this dataset there are 12 features that can be used to predict mortality by heart failure, and the target is the DEATH_EVENT column that has two values; 1 means the patient died during the follow-up period and 0 means the person still alive/dropped out of the study. The dataset consists of 300 records/rows and 12 features/columns.
And the task is Classification


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'auto-ml-exp'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code A5UU8QH44 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [5]:
from azureml.core.compute_target import ComputeTargetException
from azureml.core.compute import ComputeTarget, AmlCompute


aml_compute_cluster = "heart-compute" #can not be more than 16 chars

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=aml_compute_cluster)
    print('Found existing cluster, use it.')
except ComputeTargetException: 
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_D2_V2', min_nodes=1,max_nodes=4)
    compute_target = ComputeTarget.create(ws, aml_compute_cluster, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
print(compute_target.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 1, 'targetNodeCount': 1, 'nodeStateCounts': {'preparingNodeCount': 1, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-02-07T19:55:40.723000+00:00', 'errors': None, 'creationTime': '2021-02-07T19:53:20.699566+00:00', 'modifiedTime': '2021-02-07T19:53:36.613645+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 1, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


In [7]:

key = "HeartFailure-Dataset"

dataset = ws.datasets[key]

df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

**experiment_timeout_minutes**: 20 -> The amount of time the experiment takes before it terminates

**max_concurrent_iterations**: 4 -> The max number of iteration that can be executed in parallel 

**n_cross_validations**: 3 -> To avoid falling in overfitting

**primary_metric** : 'accuracy' -> The metric that will be optimized for model selection

**compute target**: used to run the experiment on

**task**: Classification since I will be having binary predictions

In [8]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "n_cross_validations": 3,
    "primary_metric" : 'accuracy'
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
                             compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT", 
                             enable_early_stopping= True,
                             **automl_settings
)

In [9]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Running on remote.
No run_configuration provided, running on heart-compute with default configuration
Running on remote compute: heart-compute
Parent Run ID: AutoML_da0f5073-4f86-43a2-b902-0d62d74e6569

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputati

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
from azureml.widgets import RunDetails

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [11]:
remote_run.wait_for_completion(show_output=True)



****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_da0f5073-4f86-43a2-b902-0d62d74e6569',
 'target': 'heart-compute',
 'status': 'Completed',
 'startTimeUtc': '2021-02-07T20:00:25.882047Z',
 'endTimeUtc': '2021-02-07T20:26:17.295631Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'heart-compute',
  'AMLSettingsJsonString': '{"path":null,"name":"auto-ml-exp","subscription_id":"cdbe0b43-92a0-4715-838a-f2648cc7ad21","resource_group":"aml-quickstarts-137998","workspace_name":"quick-starts-ws-137998","region":"southcentralus","compute_target":"heart-compute","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"validation_size":0.0,"n_cross_validations":3,"y_min":null,"y_max":null,"num_classes":nul

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [12]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

Package:azureml-automl-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-core, training version:1.21.0.post1, current version:1.20.0
Package:azureml-dataprep, training version:2.8.2, current version:2.7.3
Package:azureml-dataprep-native, training version:28.0.0, current version:27.0.0
Package:azureml-dataprep-rslex, training version:1.6.0, current version:1.5.0
Package:azureml-dataset-runtime, training version:1.21.0, current version:1.20.0
Package:azureml-defaults, training version:1.21.0, current version:1.20.0
Package:azureml-interpret, training version:1.21.0, current version:1.20.0
Package:azureml-pipeline-core, training version:1.21.0, current version:1.20.0
Package:azureml-telemetry, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-client, training version:1.21.0, current version:1.20.0
Package:azureml-train-automl-runtime, training version:1.21.0, current version:1.20.0


In [13]:
print('Best Run Id: ', best_run.id)
print('\n Accuracy: ', best_run_metrics['accuracy'])
print('\n Metrics: ', best_run_metrics)
print('\n Best Model: ', fitted_model)

Best Run Id:  AutoML_da0f5073-4f86-43a2-b902-0d62d74e6569_36

 Accuracy:  0.8594276094276094

 Metrics:  {'accuracy': 0.8594276094276094, 'norm_macro_recall': 0.636165722372619, 'AUC_micro': 0.8995025575621535, 'recall_score_macro': 0.8180828611863095, 'f1_score_weighted': 0.8548066059635899, 'AUC_weighted': 0.8875399537468504, 'AUC_macro': 0.8875399537468504, 'average_precision_score_micro': 0.8985135679260727, 'recall_score_micro': 0.8594276094276094, 'average_precision_score_weighted': 0.8950298873370782, 'log_loss': 0.40728120933018364, 'f1_score_macro': 0.8294613710854423, 'average_precision_score_macro': 0.8681029289591006, 'f1_score_micro': 0.8594276094276094, 'matthews_correlation': 0.6724041770619279, 'precision_score_micro': 0.8594276094276094, 'balanced_accuracy': 0.8180828611863095, 'precision_score_weighted': 0.8627820044384572, 'recall_score_weighted': 0.8594276094276094, 'weighted_accuracy': 0.8914967771530552, 'precision_score_macro': 0.8560823122376835, 'accuracy_table

In [14]:
print(fitted_model._final_estimator)

PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('18',
                                           Pipeline(memory=None,
                                                    steps=[('minmaxscaler',
                                                            MinMaxScaler(copy=True,
                                                                         feature_range=(0,
                                                                                        1))),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='gbdt',
                                                                               class_weight=None,
                                                                               colsample_bytree=0.8911111111111111,
                                                                               importa

In [15]:
#TODO: Save the best model
automl_best_model = best_run.register_model(model_name = "best-automl-model", model_path = './outputs/model.pkl', properties={'Accuracy': best_run_metrics['accuracy']})
print(automl_best_model)

Model(workspace=Workspace.create(name='quick-starts-ws-137998', subscription_id='cdbe0b43-92a0-4715-838a-f2648cc7ad21', resource_group='aml-quickstarts-137998'), name=best-automl-model, id=best-automl-model:1, version=1, tags={}, properties={'Accuracy': '0.8594276094276094'})


In [16]:
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for props_name in model.properties:
        props = model.properties[props_name]
        print ('\t',props_name, ':', props)
    print('\n')

best-automl-model version: 1
	 Accuracy : 0.8594276094276094


best_hyperdrive_model version: 1




## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [17]:
os.makedirs('./atoml-model', exist_ok=True)

best_run.download_file('/outputs/model.pkl',os.path.join('./atoml-model','best-automl-model.pkl'))

for f in best_run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./atoml-model', f.split('/')[-1])
        print(f'Downloading from {f} to {output_file_path} ...')
        best_run.download_file(name=f, output_file_path=output_file_path)

In [18]:

model=best_run.register_model(
            model_name = 'best-automl-model', 
            model_path = './outputs/model.pkl'
)

In [19]:
# Download scoring file 
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'score.py')

# Download environment file
best_run.download_file('outputs/conda_env_v_1_0_0.yml', 'environment_file.yml')

In [20]:

# Create an inference config 
inference_config = InferenceConfig(entry_script='score.py',
                                    environment=best_run.get_environment())

# Deploy the model as a web service
aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1, auth_enabled=True, enable_app_insights=True)
webservice = Model.deploy(ws, "myservice", [automl_best_model], inference_config, aci_config)
webservice.wait_for_deployment(show_output = True)


Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running......................................
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [21]:
webservice

AciWebservice(workspace=Workspace.create(name='quick-starts-ws-137998', subscription_id='cdbe0b43-92a0-4715-838a-f2648cc7ad21', resource_group='aml-quickstarts-137998'), name=myservice, image_id=None, compute_type=None, state=ACI, scoring_uri=Healthy, tags=http://5dedc2f4-430c-4728-a717-bd8f3f5eee8e.southcentralus.azurecontainer.io/score, properties={}, created_by={'hasInferenceSchema': 'True', 'hasHttps': 'False'})

In [22]:
print(webservice.state)

print(webservice.scoring_uri)

print(webservice.swagger_uri)

Healthy
http://5dedc2f4-430c-4728-a717-bd8f3f5eee8e.southcentralus.azurecontainer.io/score
http://5dedc2f4-430c-4728-a717-bd8f3f5eee8e.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [23]:
#test data

test_df = df.sample(5) 
label_df = test_df.pop('DEATH_EVENT')

test_sample = json.dumps({'data': test_df.to_dict(orient='records')})

print(test_sample)

{"data": [{"age": 70.0, "anaemia": 0, "creatinine_phosphokinase": 66, "diabetes": 1, "ejection_fraction": 45, "high_blood_pressure": 0, "platelets": 249000.0, "serum_creatinine": 0.8, "serum_sodium": 136, "sex": 1, "smoking": 1, "time": 80}, {"age": 70.0, "anaemia": 0, "creatinine_phosphokinase": 88, "diabetes": 1, "ejection_fraction": 35, "high_blood_pressure": 1, "platelets": 236000.0, "serum_creatinine": 1.2, "serum_sodium": 132, "sex": 0, "smoking": 0, "time": 215}, {"age": 49.0, "anaemia": 0, "creatinine_phosphokinase": 789, "diabetes": 0, "ejection_fraction": 20, "high_blood_pressure": 1, "platelets": 319000.0, "serum_creatinine": 1.1, "serum_sodium": 136, "sex": 1, "smoking": 1, "time": 55}, {"age": 82.0, "anaemia": 0, "creatinine_phosphokinase": 70, "diabetes": 1, "ejection_fraction": 30, "high_blood_pressure": 0, "platelets": 200000.0, "serum_creatinine": 1.2, "serum_sodium": 132, "sex": 1, "smoking": 1, "time": 26}, {"age": 50.0, "anaemia": 1, "creatinine_phosphokinase": 1051

In [24]:
output = webservice.run(test_sample)
print(output)

{"result": [0, 0, 1, 1, 0]}


In [41]:
# Send a request to the web service you deployed to test it.

# import requests

# headers = {'Content-type': 'application/json'}

# response = requests.post(webservice.scoring_uri, test_sample, headers=headers)

In [48]:
# print(response.text)

In [25]:
print(label_df)

90     0
253    0
58     1
29     1
278    0
Name: DEATH_EVENT, dtype: int64


TODO: In the cell below, print the logs of the web service and delete the service

In [26]:
print(webservice.get_logs())

2021-02-07T20:46:13,282251601+00:00 - iot-server/run 
2021-02-07T20:46:13,285338642+00:00 - rsyslog/run 
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_20a8278aa8b20dd48cc50f56a6d2586c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-07T20:46:13,288214480+00:00 - nginx/run 
2021-02-07T20:46:13,293174445+00:00 - gunicorn/run 
rsyslogd

In [49]:
webservice.delete()